# Quantile Matching and LightGBM

This notebook showcases the use of quantile matching paired with LightGBM quantile regression to predict probability distributions.

To install the needed dependencies, follow the instructions in the README at the root of the repository.



In [ ]:
from abc import ABC, abstractmethod

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from lightgbm import LGBMRegressor
from scipy import optimize, stats
from scipy.interpolate import PchipInterpolator
from sklearn.datasets import load_diabetes


## Implementation

The goal of quantile matching is to estimate a distribution function given a sample of quantile values.

The curve doesn't have to perfectly fit the quantiles, but rather be as close as possible, while keeping the properties that make it a distribution function.
Specifically, we are interested in estimating the inverse cumulative distribution function: given a probability `alpha`, we want to know what is the value `v` for which `P(X<v)=alpha`, where `P` represent a probability and `X` the random variable. This can be used to represent uncertainty and estimate prediction intervals.

Let's define 3 alternatives to estimate such function from a set of quantiles:
- Fit a Normal Distribution
- Fit a Half Normal Distributio
- Cubic Interpolation: use cubic splines to estimate a smooth increasing curve.

To do that, we use an easy-to-extend design pattern: 
- a base abstract class which defines an interface
- a set of concrete classes which implement the different methods
- a factory that returns the class for the desired method

In [ ]:
class QuantileMatcherBase(ABC):
    @abstractmethod
    def fit_one(self, alphas, quant_values):
        pass

    @abstractmethod
    def predict_one(self, alphas):
        pass


def quantile_matcher_factory(match, **kwargs) -> QuantileMatcherBase:
    matcher_map = {
        "normal": QuantileMatcherNormCurvFit,
        "half_normal": QuantileMatcherHalfNormCurvFit,
        "cubic_interpolation": QuantileMatcherCubicInterpolation,
    }
    if match not in matcher_map:
        raise ValueError(f"Unknown matcher {match}")

    return matcher_map[match](**kwargs)


class QuantileMatcherNormCurvFit(QuantileMatcherBase):
    """Normal distribution quantile matcher."""

    def __init__(self):
        self.params = None

    def fit_one(self, alphas, quant_values):
        self.params, _ = optimize.curve_fit(
            lambda x, mu, sigma: stats.norm.isf(x, mu, sigma),
            alphas,
            1 - quant_values,
        )

    def predict_one(self, alphas):
        return 1 - stats.norm.isf(alphas, *self.params)


    
    
class QuantileMatcherHalfNormCurvFit(QuantileMatcherBase):
    """Half-Normal distribution quantile matchers."""

    def __init__(self):
        self.below = QuantileMatcherNormCurvFit()
        self.above = QuantileMatcherNormCurvFit()

    def fit_one(self, alphas, quant_values):
        self.below.fit_one(alphas[alphas<=0.5],quant_values[alphas<=0.5])
        self.above.fit_one(alphas[alphas>=0.5],quant_values[alphas>=0.5])
        
        mu = (self.below.params[0] + self.above.params[0]) / 2
        self.below.params[0] = mu
        self.above.params[0] = mu

    def predict_one(self, alphas):
        pred = self.above.predict_one(alphas)
        pred_below = self.below.predict_one(alphas)
        pred[alphas<0.5] = pred_below[alphas<0.5]
        return pred
    
    

class QuantileMatcherCubicInterpolation(QuantileMatcherBase):
    """Increasing cubic interpolation quantile matcher."""
    def __init__(self):
        self.params = None

    def fit_one(self, alphas, quant_values):
        self.interp = PchipInterpolator(alphas, quant_values)

    def predict_one(self, alphas):
        return self.interp(alphas)

Now, let's define a class that estimates a set of quantiles, and uses the methods above to predict them on a fine grid.

For this, we are going to use LightGBM quantile regression.

In [ ]:
class ProbLGBMRegressor:
    _forbidden_keys = (
        "objective",
        "objective_type",
        "app",
        "application",
        "loss",
        "alpha",
    )

    def __init__(
        self, 
        alphas=np.array([0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]), 
        **lgbm_args
    ):
        self.alphas = alphas

        for key in self._forbidden_keys:
            if key in lgbm_args:
                raise ValueError(f"{key} parameter is not allowed.")

        self._models = {}
        for alpha in self.alphas:
            self._models[alpha] = LGBMRegressor(
                objective="quantile", alpha=alpha, **lgbm_args
            )

    def fit(self, x, y):
        for alpha in self.alphas:
            self._models[alpha].fit(x, y)

    def predict_raw(self, x):
        return pd.DataFrame(
            {alpha: model.predict(x) for alpha, model in self._models.items()}
        )

    def predict_cdf(
        self,
        x,
        inference_alphas=np.linspace(0.001, 0.999, 999),
        match="normal_curve_fit",
        **matcher_params,
    ):
        # Compute predictions for the limited set of quantiles.
        raw_preds = self.predict_raw(x)

        # Estimate.
        matcher = quantile_matcher_factory(match, **matcher_params)
        predictions = []
        for _, row in raw_preds.iterrows():
            matcher.fit_one(self.alphas, row.values)
            preds = matcher.predict_one(inference_alphas)
            predictions.append(preds)

        return pd.DataFrame(predictions, columns=inference_alphas)

## Usage Example

To showcase quantile matching, let's use the diabetes dataset: 10 features are used to predict a target that quantifies the development of diabetes.

In [ ]:
x,y = load_diabetes(return_X_y=True, as_frame=True)

In [ ]:
x.head()

In [ ]:
# Fit a regressor
prob_lgbm = ProbLGBMRegressor()
prob_lgbm.fit(x,y)

In [ ]:
%%time
# Predict the distributions with all methods
predicted_cdf = {}
for match in ["normal","half_normal","cubic_interpolation"]:
    predicted_cdf[match] = prob_lgbm.predict_cdf(x, match=match)

## Visualization

Let's visualize these distributions:

In [ ]:
# For visualization purposes, we predict also the "raw" values
predicted_raw = prob_lgbm.predict_raw(x)

In [ ]:
def get_fig_cumulative_distribution_function(predicted_cdf, predicted_raw, idx):
    # Small artifact to ensure same range in figures
    max_limit = max([pred.iloc[idx, -1] for pred in predicted_cdf.values()]) + 5
    min_limit = max([pred.iloc[idx, 0] for pred in predicted_cdf.values()]) - 5

    # Create traces for each distribution.
    trace = []
    for match, pred_cdf in predicted_cdf.items():
        x = [min_limit] + list(pred_cdf.iloc[idx].values) + [max_limit]
        y = [0] + list(pred_cdf.columns) + [1]
        trace.append(go.Scatter(x=x, y=y, mode="lines", name=match.title()))

    # Add trace for raw quantile predictions.
    trace.append(
        go.Scatter(
            x=predicted_raw.iloc[idx],
            y=predicted_raw.columns,
            mode="markers",
            name="Raw Predictions",
            marker={"size": 10},
        )
    )

    # Create the figure
    fig = go.Figure(trace)
    fig.update_layout(
        title="Cumulative Distribution Functions",
        yaxis_title="alpha",
        xaxis_title="quantile",
    )
    # Set x-axis limits
    fig.update_xaxes(range=(min_limit, max_limit))
    return fig


def get_fig_probability_distribution_function(predicted_cdf, idx):
    trace = []

    for match, pred_cdf in predicted_cdf.items():
        quantiles = pred_cdf.iloc[idx].values
        icdf_values = pred_cdf.columns.values

        # Estimate the PDF using finite differences
        diff_icdf = np.diff(icdf_values)
        diff_quantiles = np.diff(quantiles)
        pdf_est = diff_icdf / diff_quantiles

        # Create a Plotly figure for the estimated PDF
        trace.append(
            go.Scatter(
                x=quantiles[:-1],
                y=pdf_est,
                mode="lines",
                fill="tozeroy",
                name=match,
            )
        )

    fig = go.Figure(data=trace)

    # Add labels and title
    fig.update_layout(
        xaxis_title="quantile",
        yaxis_title="probability density",
        title="Estimated Probability Density Function",
    )
    return fig


In [ ]:
# Let's visualize the cumulative and probability distribution for the first few predictions.
for idx in range(5):
    get_fig_cumulative_distribution_function(predicted_cdf, predicted_raw, idx).show()
    get_fig_probability_distribution_function(predicted_cdf, idx).show()

We can see that the normal and the half-normal distribution don't coincide, suggesting an asymmetry in the true underlying distribution. 

We also notice that the cubic interpolation often gives multi-modal extreme results. This is due to the fact that interpolation is not constrained and tends to have high derivative when fitting close points. These results are probably not realistic, and a smoothing technique might help in this setting.

All in all, the half-normal distribution might be the best choice, as it provides a realistic distribution while being able to model asymmetric behaviours. 
However, the best way to choose the matchin algorithm would be to cross-validate predictions and compute some relevant metrics, such as width of the prediction intervals combined with their accuracy (a 90% interval should contain the prediction 90% of the time).